<a href="https://colab.research.google.com/github/rakib06/FileShare/blob/main/train_LIVENESS_MODEL_30_AUG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/jomariya23156/face-recognition-with-liveness-web-login.git

Cloning into 'face-recognition-with-liveness-web-login'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 173 (delta 23), reused 86 (delta 7), pack-reused 65
Receiving objects: 100% (173/173), 74.07 MiB | 22.97 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [ ]:
#!python /content/face-recognition-with-liveness-web-login/face_recognition_and_liveness/face_liveness_detection/train_model.py -d /content/drive/MyDrive/livenessnet/dataset -m /content/drive/MyDrive/livenessnet/face_detector -l /content/drive/MyDrive/livenessnet/le.pickle

In [2]:
import os
os.chdir('/content/face-recognition-with-liveness-web-login/face_recognition_and_liveness/face_liveness_detection')

In [3]:
# set the matplotlib backend, so figures can be saved in the background
import matplotlib
matplotlib.use('Agg') # Agg is used for writing files

from livenessnet import LivenessNet # our model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

In [8]:
args = {}
args["dataset"] = '/content/drive/MyDrive/livenessnet/dataset'

In [9]:
f'fake: {len(list(paths.list_images(args["dataset"]+"/fake")))}'

'fake: 627'

In [10]:
f'real: {len(list(paths.list_images(args["dataset"]+"/real")))}'

'real: 799'

In [11]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"]))
#print("imagePaths: ", imagePaths)
data = []
labels = []

[INFO] loading images...


In [12]:
%%timeit

import tqdm
for imagePath in tqdm.tqdm(imagePaths):
    # extract the class label from the filename, load the image and
    # resize it to be a fixed 32x32 pixels, ignoring aspect ratio
    label = imagePath.split(os.path.sep)[-2]
    # print(": ", imagePath.split(os.path.sep)[1])
    # print("os.path.sep: ", os.path.sep)
    # print("label: ", label)
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (64, 64))

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

100%|██████████| 1426/1426 [00:03<00:00, 367.59it/s]

3.69 s ± 175 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
len(data)


11408

In [14]:
# convert to ndarray and do feature scaling
# tf works best with ndarray and it's super fast and efficient!
data = np.array(data, dtype='float') / 255.0

# encode the labels from (fake, real) to  (0,1)
# and do one-hot encoding
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = tf.keras.utils.to_categorical(labels, 2) # one-hot encoding


In [15]:
len(data)

11408

In [ ]:
# train/test split
# we go for traditional 80/20 split
# Theoretically, we have small dataset we need test set to be a bit bigger
# 75/25 or 70/30 split would be ideal, but from the trial and error
# 80/20 gives a better result, so we go for it
# Another thing to consider, since my dataset has only about 14 images of faces from card/solid image
# so 80/20 has a higher chance that those images will be in training set rather than test set (and none on training set)
# X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, random_state=42)
# (X_train, X_test, y_train, y_test) = train_test_split(data, labels,
#                                                   test_size=0.20, random_state=42)


In [16]:
aug = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode='nearest')


In [17]:
# build a model
# define hyperparameters
INIT_LR = 1e-4 # initial learning rate
BATCH_SIZE = 4


In [18]:
# we don't need early stopping here because we have a small dataset, there is no need to do so
# initialize the optimizer and model
print('[INFO] compiling model...')
optimizer = tf.keras.optimizers.Adam(learning_rate=INIT_LR)


[INFO] compiling model...


In [19]:
model = LivenessNet.build(width=64, height=64, depth=3, classes=len(le.classes_))


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 16)       64        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 16)        2320      
                                                                 
 batch_normalization_1 (Batc  (None, 64, 64, 16)       64        
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 16)       0         
 )                                                               
                                                        

In [21]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [22]:
# early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)


In [23]:
(X_train, X_test, y_train, y_test) = train_test_split(data, labels,
                                                  test_size=0.20, random_state=42)


In [24]:
# train the model
EPOCHS = 20
print(f'[INFO] training model for {EPOCHS} epochs...')
history = model.fit(x=aug.flow(X_train, y_train, batch_size=BATCH_SIZE),
                    validation_data=(X_test, y_test),
                    steps_per_epoch=len(X_train) // BATCH_SIZE,
                    epochs=EPOCHS)

[INFO] training model for 20 epochs...
Epoch 1/20
2281/2281 [==============================] - 43s 12ms/step - loss: 0.6629 - accuracy: 0.6796 - val_loss: 0.4114 - val_accuracy: 0.8611
Epoch 2/20
2281/2281 [==============================] - 28s 12ms/step - loss: 0.5186 - accuracy: 0.7643 - val_loss: 0.2529 - val_accuracy: 0.8988
Epoch 3/20
2281/2281 [==============================] - 28s 12ms/step - loss: 0.4645 - accuracy: 0.7966 - val_loss: 0.2912 - val_accuracy: 0.8865
Epoch 4/20
2281/2281 [==============================] - 28s 12ms/step - loss: 0.4390 - accuracy: 0.8083 - val_loss: 0.5338 - val_accuracy: 0.8019
Epoch 5/20
2281/2281 [==============================] - 28s 12ms/step - loss: 0.4210 - accuracy: 0.8205 - val_loss: 0.2203 - val_accuracy: 0.9229
Epoch 6/20
2281/2281 [==============================] - 28s 12ms/step - loss: 0.3940 - accuracy: 0.8319 - val_loss: 0.2723 - val_accuracy: 0.9268
Epoch 7/20
2281/2281 [==============================] - 28s 12ms/step - loss: 0.3754 

In [25]:
result = model.evaluate(X_test, y_test, batch_size=10000)
print('\nTest Acc:', result[1])


1/1 [==============================] - 3s 3s/step - loss: 0.1861 - accuracy: 0.9207

Test Acc: 0.9206836223602295


In [29]:
import time
folder = f'/content/drive/MyDrive/livenessnet/model-E{EPOCHS}-Acc-{int(result[1]*100)}-t-{int(time.time())}'
os.mkdir(folder)

model.save(f"{folder}/liveness.model")



In [30]:
model.save_weights(f'{folder}/livenessnet_weights.h5')
f = open(f"{folder}/encoded_faces.pickle", "wb")
f.write(pickle.dumps(le))
f.close()

In [31]:
# model.save_weights(f'{folder}/livenessnet_weights.h5')
f = open(f"{folder}/label_encoder.pickle", "wb")
f.write(pickle.dumps(le))
f.close()